<a href="https://colab.research.google.com/github/DenisSud/aiijc-701w/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [4]:
MODEL_NAME = "Qwen/Qwen3-1.7B"
DATA_PATH = "/content/drive/MyDrive/train.csv"
OUTPUT_DIR = "/content/drive/MyDrive/gspo_math_model/"

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 701,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-05 10:35:45 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-05 10:35:45 [__init__.py:239] Automatically detected platform cuda.
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.54.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-1.7B with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 1

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
import gc
gc.collect()

0

### Data Prep
<a name="Data"></a>

In [22]:
from datasets import load_dataset
# Load the dataset
dataset = load_dataset(
    "csv",
    data_files={"train": DATA_PATH},
    split="train"
)

In [25]:
dataset[0]["task"]

'The value of $y$ varies inversely as $\\sqrt x$ and when $x=24$, $y=15$. What is $x$ when $y=3$?'

In [26]:
dataset[0]["answer"]

'[600]'

In [27]:
from fractions import Fraction

# Define your system prompt
system_prompt = "You are a helpful assistant that solves math problems step by step."

def parse_answer(s):
    """Extracts answer from various formats, returns float."""
    if s is None:
        return None

    s = str(s)

    # Try different answer formats in order of preference
    patterns = [
        r"\[(\d+(?:\.\d+)?)\]",           # [52] format
        r"\\boxed\{(\d+(?:\.\d+)?)\}",    # \boxed{52} format
        r"boxed\{(\d+(?:\.\d+)?)\}",      # boxed{52} format
    ]

    for pattern in patterns:
        match = re.search(pattern, s, re.IGNORECASE)
        if match:
            val = match.group(1).strip()
            break
    else:
        return None

    try:
        return float(val)
    except ValueError:
        try:
            return float(Fraction(val))
        except (ValueError, ZeroDivisionError):
            return None

def add_instruction(example):
    # Create the prompt content
    prompt_content = (
        "Please solve the following problem step by step:\n"
        f"Problem: {example['task'].strip()}\n\n"
        "1. First, understand what is being asked\n"
        "2. Show your reasoning\n"
        "3. Provide your final answer in brackets like [52]\n\n"
        "Your response should end with your final numerical answer in brackets with no other characters."
    )

    # Return the chat format structure
    return {
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt_content},
        ],
        "answer": parse_answer(example["answer"]),
    }

# Apply the transformation
dataset = dataset.map(add_instruction)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
dataset[0]["prompt"]

[{'content': 'You are a helpful assistant that solves math problems step by step.',
  'role': 'system'},
 {'content': 'Please solve the following problem step by step:\nProblem: The value of $y$ varies inversely as $\\sqrt x$ and when $x=24$, $y=15$. What is $x$ when $y=3$?\n\n1. First, understand what is being asked\n2. Show your reasoning\n3. Provide your final answer in brackets like [52]\n\nYour response should end with your final numerical answer in brackets with no other characters.',
  'role': 'user'}]

In [20]:
dataset[0]["answer"]

600.0

In [28]:
def get_max_prompt_length(dataset, tokenizer):
    """Find maximum token length of prompts in the dataset"""
    max_length = 0
    max_example = None

    # Process each example to find longest prompt
    for i, example in enumerate(dataset):
        # Format the prompt using chat template
        prompt = tokenizer.apply_chat_template(
            example["prompt"],
            tokenize=False,  # Get string first for length analysis
            add_generation_prompt=True  # Critical for correct prompt formatting
        )

        # Tokenize and get length
        tokenized = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=False,
            padding=False
        )
        length = tokenized.input_ids.shape[1]

        # Track maximum
        if length > max_length:
            max_length = length
            max_example = {
                "index": i,
                "prompt": prompt,
                "token_length": length
            }

    return max_length, max_example

# Now find max length
maximum_length, max_example = get_max_prompt_length(dataset, tokenizer)


In [29]:
max_example

{'index': 525,
 'prompt': '<|im_start|>system\nYou are a helpful assistant that solves math problems step by step.<|im_end|>\n<|im_start|>user\nPlease solve the following problem step by step:\nProblem: There is a total of 70 squares of three sizes whose vertices are points on this rectangular $3\\times n$ grid of points. What is the value of $n$?\n\n[asy]\n\nunitsize(0.4 inch);\n\ndot((0,0),linewidth(9bp));\ndot((1,0),linewidth(9bp));\ndot((2,0),linewidth(9bp));\ndot((0,1),linewidth(9bp));\ndot((0,2),linewidth(9bp));\ndot((1,1),linewidth(9bp));\ndot((2,1),linewidth(9bp));\ndot((1,2),linewidth(9bp));\ndot((2,2),linewidth(9bp));\n\nfilldraw((2.95,-0.05)--(3.05,-0.05)--(3.05,0.05)--(2.95,0.05)--cycle,black);\nfilldraw((2.45,-0.05)--(2.55,-0.05)--(2.55,0.05)--(2.45,0.05)--cycle,black);\nfilldraw((3.45,-0.05)--(3.55,-0.05)--(3.55,0.05)--(3.45,0.05)--cycle,black);\n\nfilldraw((2.95,0.95)--(3.05,0.95)--(3.05,1.05)--(2.95,1.05)--cycle,black);\nfilldraw((2.45,0.95)--(2.55,0.95)--(2.55,1.05)--(

In [30]:
def reward_func(prompts, completions, **kwargs):
    """Reward based on negative absolute error to true answer."""
    rewards = []
    for prompt, completion in zip(prompts, completions):
        # Extract the answer from the original dataset by finding the example
        # that matches this prompt
        matching_example = None
        for example in dataset:
            if example["prompt"] == prompt:
                matching_example = example
                break

        if matching_example is None:
            rewards.append(-1.0)
            continue

        # Debug: print what we're trying to parse
        print(f"Raw answer from dataset: {repr(matching_example['answer'])}")
        print(f"Completion: {repr(completion[:100])}...")  # First 100 chars

        true_val = parse_answer(matching_example["answer"])
        pred = parse_answer(completion)

        print(f"Parsed true_val: {true_val}, pred: {pred}")

        if pred is None or true_val is None:
            rewards.append(-1.0)
        else:
            rewards.append(-abs(pred - true_val))
    return rewards

Let's look at the first row:

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [34]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    fp16_full_eval = True,
    per_device_eval_batch_size = 4,
    eval_accumulation_steps = 1,
    eval_strategy = "steps",
    eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [35]:
# For optional training + evaluation
new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
      reward_func
    ],
    args = training_args,
    # train_dataset = dataset,

    # For optional training + evaluation
    train_dataset = new_dataset["train"],
    eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 990 | Num Epochs = 1 | Total steps = 990
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Raw answer from dataset: 774.0
Completion: [{'role': 'assistant', 'content': "<think>\nOkay, let's see. The problem is 459 plus 315. I need to add these two numbers. Alright, first step is to understand what the question is asking. It's a straightforward addition problem, right? So, I just need to add 459 and 315.\n\nHmm, let me think about how to add these numbers. Maybe I can break it down into smaller parts. Let's start with the units place. The ones digit of 459 is 9, and the ones digit of 315 is 5. Adding those together: 9 + 5 equals 14. So, I'll write down 4 in the ones place and carry over 1 to the tens place.\n\nNext, the tens place. The tens digit of 459 is 5, and the tens digit of 315 is 1. Adding them with the carry-over of 1: 5 + 1 + 1 (from the carry-over) equals 7. So, the tens place becomes 7.\n\nNow the hundreds place. The hundreds digit of 459 is 4, and the hundreds digit of 315 is 3. Adding them together: 4 + 3 equals 7. So, the hundreds place is 7.\n\nPutting it all 

Step,Training Loss,Validation Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,entropy,rewards / reward_func / mean,rewards / reward_func / std


Raw answer from dataset: 21.0
Completion: [{'role': 'assistant', 'content': "<think>\nOkay, let's see. There are two intersecting circles with a common chord of 16 feet. Their centers are on opposite sides of the chord, and the radii are 10 ft and 17 ft. I need to find the distance between the centers. Hmm, how do I approach this?\n\nFirst, I remember that when two circles intersect, the line connecting their centers is perpendicular to the common chord and bisects it. So, if the chord is 16 feet long, then each half of the chord is 8 feet. Let me draw a diagram in my mind. The common chord is 16 ft, so half of that is 8 ft. The centers of the circles are on opposite sides of this chord, which means if one center is above the chord, the other is below. \n\nLet me denote the centers as O1 and O2. The distance between them is what I need to find. Let's call the midpoint of the chord AB, where AB is 16 ft. So, the midpoint is point M. Then AM = MB = 8 ft. \n\nSince O1 and O2 are on opposi

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 990 | Num Epochs = 1 | Total steps = 1,320
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 1 x 1) = 3
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Raw answer from dataset: 774.0
Completion: [{'role': 'assistant', 'content': "<think>\nOkay, let's see. The problem is to add 459 and 315. Alright, so I need to figure out what 459 plus 315 is. Let me start by breaking it down.\n\nFirst, I remember that adding numbers can be done by adding the ones place, then the tens, then the hundreds. Let me try that. So, let's look at the numbers:\n\n459\n+315\n------\n\nStarting with the ones place: 9 (from 459) plus 5 (from 315) equals 14. Hmm, so I write down 4 and carry over 1 to the tens place. \n\nNext, the tens place. The original numbers are 5 (tens in 459) and 1 (tens in 315). Adding the tens: 5 + 1 is 6, plus the carried over 1 makes 7. So the tens place is 7.\n\nNow the hundreds place: 4 (from 459) and 3 (from 315) add up to 7. So that's 7 in the hundreds place.\n\nPutting it all together, the sum should be 774. Wait, let me check that again. Because 459 + 315. Let me try another way to verify.\n\nAlternatively, I could add them vertica

RuntimeError: shape '[-1, 4]' is invalid for input of size 3

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

' - Answers\nEducation\nMath and Arithmetic\nWhat is the sqrt of 101?\nWiki User\n∙ 2009-06-05 01:02:40\nBest Answer\nCopy\n10.05\nWiki User\n∙ 2009-06-05 01:02:40\nThis answer is:\n🙏\n0\n🤨\n0\n😮\n0\nWhat is the sqrt of 100?\nThe square root of 100 is 10; 10x10=100\nWhat is the Sqrt of 100?\nSqrt (100) = +10 &amp; -10 since both (+10) X (+10) and (-10) X (-\n10) equal 100\nWhat is the sqrt of 10000?\nIt is 100.\nHow many meters in a kilometer?\nThere are 1000 meters in a kilometer. 100 centimeter in a\nmeter. 1,000 meters in a km. 100cm in a meter.\nThe square root 100 is?\n-10 +10\nSqrt x equals 7 sqrt 2 find the value of x?\nSqrt(x) = 7 Sqrt(2) x = 72 x 2 x = 49 x 2 x = 98\nSqrt 100 simplified?\nSqrt(100) = 10 is its simplest form.\nWhat is the sqr root of 100?\n10 or -10 because 10*10=100\nand (-10)(-10)=100\nSqr\'s and cubs?\nSqrt(100) = 10 and 102 = 100Sqrt(1000) = 31.6 and 31.62 = 10001Sqrt(24) = 4.9 and 4.92 = 24If you are unsure, just check them out by calculator.The "cube" of 

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, so I need to find the square root of 101. Hmm, let me think. I remember that square roots of numbers between perfect squares are irrational, meaning they can't be expressed as simple fractions. Let's see, the perfect squares around 101 are 100 and 121. Since 10² is 100 and 11² is 121, the square root of 101 must be between 10 and 11. \n\nWait, but how close is it to 10 or 11? Maybe I can estimate it a bit more. Let me try multiplying 10.05 by itself. Calculating (10.05)² would be 10.05 * 10.05. Let's break that down: 10 * 10 = 100, 10*0.05=0.5, 0.05*10=0.5, and 0.05*0.05=0.0025. Adding those up: 100 + 0.5 + 0.5 + 0.0025 = 101.0025. That's a bit more than 101. So maybe 10.05 is too high. \n\nLet me try 10.04... Let's compute (10.04)². That would be 10.04 * 10.04. Breaking it down: 100 + 40*0.04=1.6 + 0.04*0.04=0.0016. So 100 + 1.6 + 0.0016 = 101.6016. Still a bit high. So maybe 10.03. Let's try (10.03)²: 100 + 30*0.03=0.9 + 0.03*0.03=0.0009. Adding gives 100 + 0.9 + 0.0009 = 100.

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged(OUTPUT_DIR, tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("densud2/Qwen3-1.7B-Math", tokenizer, save_method = "merged_16bit")

# Merge to 4bit
if False: model.save_pretrained_merged(OUTPUT_DIR, tokenizer, save_method = "merged_4bit",)
if True: model.push_to_hub_merged("densud2/Qwen3-1.7B-Math", tokenizer, save_method = "merged_4bit")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
